In [1]:
import pickle
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.externals import joblib
import numpy as np
from pandas import DataFrame
from IPython.display import display

In [2]:
def display_metrics(true_labels, predicted_labels, labels):
    print("Accuracy:", accuracy_score(true_labels, predicted_labels))
    print("\nConfusion matrix")
    display(DataFrame(confusion_matrix(true_labels, predicted_labels),
                      index=["Actual " + label for label in labels], columns=["Predicted " + label for label in labels]))
    print("\nPrecision, recall, F-score, support")
    display(DataFrame(np.array(precision_recall_fscore_support(true_labels, predicted_labels)),
            index=("precision", "recall", "F-score", "support"), columns=labels))

# SVM Classifier with features extracted from the last layer of VGG-19

The general idea here is to use transfer learning approach. Inputs to the model are image representations in the form of vectors of length 4096, which are obtained after propagating images through a VGG-19 model pretrained on ImageNet and taking the outputs of its last layer before classification.

Having gotten the features, we need to classify the images into two classes: "good" cars and "damaged" cars. Here we'll try to use an SVM classifier, as it is suitable for binary classification of objects represented as feature vectors.

In [3]:
with open("train_vgg_19.p", "rb") as train_pickle:
    train_cars, train_labels = pickle.load(train_pickle)

In [4]:
print(train_cars[0], train_cars[0].shape)
print(train_labels[0])

[0.22569361 0.         0.4772556  ... 0.04154909 0.61163104 0.07535654] (4096,)
1


Let's begin with SVM classifier with a default RBF kernel.

In [5]:
clf_rbf = svm.SVC()
clf_rbf.fit(train_cars, train_labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
predicted_labels = clf_rbf.predict(train_cars)

In [7]:
display_metrics(train_labels, predicted_labels, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9551734725207246

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,3018,180
Actual Damaged Cars,112,3204



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.964217,0.946809
recall,0.943715,0.966224
F-score,0.953856,0.956418
support,3198.000000,3316.000000


Actually, it looks not that bad. Let's see the results for the validation set.

In [8]:
with open("valid_vgg_19.p", "rb") as valid_pickle:
    valid_cars, valid_labels = pickle.load(valid_pickle)

In [9]:
predicted_labels_valid = clf_rbf.predict(valid_cars)

In [10]:
display_metrics(valid_labels, predicted_labels_valid, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9557739557739557

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,374,19
Actual Damaged Cars,17,404



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.956522,0.955083
recall,0.951654,0.959620
F-score,0.954082,0.957346
support,393.000000,421.000000


The results seem consistent.

Now let's train a linear SVM classifier for comparison.

In [11]:
clf_lin = svm.SVC(kernel='linear')
clf_lin.fit(train_cars, train_labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
predicted_labels = clf_lin.predict(train_cars)

In [13]:
display_metrics(train_labels, predicted_labels, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9983113294442739

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,3192,6
Actual Damaged Cars,5,3311



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.998436,0.998191
recall,0.998124,0.998492
F-score,0.998280,0.998342
support,3198.000000,3316.000000


The results are too good to be true. Let's check for the validation dataset.

In [14]:
predicted_labels_valid = clf_lin.predict(valid_cars)

In [15]:
display_metrics(valid_labels, predicted_labels_valid, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9766584766584766

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,386,7
Actual Damaged Cars,12,409



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.969849,0.983173
recall,0.982188,0.971496
F-score,0.975980,0.977300
support,393.000000,421.000000


The results are again consistent and better than for RBF kernel. Also the linear classifier has an advantage of being simpler.

Let's save the trained linear SVM classifier.

In [16]:
joblib.dump(clf_lin, 'svm_vgg_19.pkl')

['svm_vgg_19.pkl']